In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
median_path = 'Final/median_sale_price.csv'
median_df = pd.read_csv(median_path)

median_df.drop(["SizeRank", "RegionType", "RegionID", "StateName"], axis=1, inplace=True)
median_df.dropna(axis=0, inplace=True)
median_df.set_index("RegionName", inplace=True)
new_columns = pd.to_datetime(median_df.columns)
new_columns = new_columns
median_df.columns = new_columns
median_df = median_df.T
median_df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'Final/median_sale_price.csv'

In [ ]:
mean_path = 'Final/mean_sale_price.csv'
mean_df = pd.read_csv(mean_path)

mean_df.drop(["SizeRank", "RegionType", "RegionID", "StateName"], axis=1, inplace=True)
mean_df.dropna(axis=0, inplace=True)
mean_df.set_index("RegionName", inplace=True)
new_columns = pd.to_datetime(mean_df.columns)
new_columns = new_columns
mean_df.columns = new_columns
mean_df = mean_df.T
mean_df.head(3)

In [ ]:
income_path = "Final/income_data.csv"
income_df = pd.read_csv(income_path)

income_df.columns = ["Date", "Income"]
income_df.index = pd.to_datetime(income_df["Date"])
income_df.drop(["Date"], axis=1, inplace=True)
income_df = income_df[income_df.index.year >= 2008]
income_df

In [ ]:
nrows = 2
ncols = 1
total_plots = nrows * ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 15))

# Adjust x_values and income_x_values to be the row index (dates)
x_values = median_df.index
income_x_values = income_df.index

# Plot each location's data in the grid
for i, col in enumerate(median_df.columns[:total_plots]):
    if nrows == 1 or ncols == 1:
        ax = axes[i]
    else:
        ax = axes[i // ncols, i % ncols]
    
    # Plot median data for the current location
    ax.plot(x_values, median_df[col], label='Median Sale Price')

    # Plot corresponding income data for the current location
    # Assuming the income_df has the same locations as columns
    ax.plot(income_x_values, income_df["Income"].values, label='Median Income')
    
    ax.set_title(col)
    ax.legend()

plt.tight_layout()
plt.show()

compare with inflation, look at average rate of change from year to year for both datasets, make comparisons by state.


In [ ]:
def calculate_annual_inflation(row):
    monthly_rates_decimal = [1 + (rate / 100) for rate in row]
    annual_rate = np.prod(monthly_rates_decimal)
    annual_inflation_rate = (annual_rate - 1) * 100
    return annual_inflation_rate

inflation_path = "Final/inflation_data.csv"

inflation_df = pd.read_csv(inflation_path)
inflation_df.drop(["HALF1", "HALF2"], axis=1, inplace=True)
inflation_df["Annual"] = inflation_df.iloc[:, 1:].apply(calculate_annual_inflation, axis=1)
inflation_df['CumulativeInflation'] = (1 + inflation_df['Annual'] / 100).cumprod()
base_year = 2008
base_cpi = inflation_df[inflation_df['Year'] == base_year]['CumulativeInflation'].values[0]
inflation_df['CumulativeInflation'] /= base_cpi
inflation_df.head()

In [ ]:
normalized_mean_df = mean_df.copy()
normalized_median_df = median_df.copy()
normalized_income_df = income_df.copy()

cumulative_inflation_dict = inflation_df.set_index('Year')['CumulativeInflation'].to_dict()

# Normalize mean_df
for index in mean_df.index:
    for col in mean_df.columns[2:]:  # Assuming the first two columns are not datetime columns
        # Extract the year from the datetime column name
        if isinstance(col, pd.Timestamp):
            year = col.year
            # Normalize the price using the cumulative inflation index for that year
            normalized_mean_df.at[index, col] = mean_df.at[index, col] / cumulative_inflation_dict[year]

# Normalize median_df
for index in median_df.index:
    for col in median_df.columns[2:]:  # Assuming the first two columns are not datetime columns
        # Extract the year from the datetime column name
        if isinstance(col, pd.Timestamp):
            year = col.year
            # Normalize the price using the cumulative inflation index for that year
            normalized_median_df.at[index, col] = median_df.at[index, col] / cumulative_inflation_dict[year]

# Normalize income_df
for col in income_df.columns:
    if isinstance(col, pd.Timestamp):
        year = col.year
        normalized_income_df[col] = income_df[col] / cumulative_inflation_dict[year]

median_df = normalized_median_df
income_df = normalized_income_df

In [ ]:
nrows = 3
ncols = 1
total_plots = nrows * ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 15))

# Adjust x_values and income_x_values to be the row index (dates)
x_values = normalized_median_df.index
income_x_values = normalized_income_df.index

# Plot each location's data in the grid
for i, col in enumerate(normalized_median_df.columns[:total_plots]):
    if nrows == 1 or ncols == 1:
        ax = axes[i]
    else:
        ax = axes[i // ncols, i % ncols]
    
    # Plot median data for the current location
    ax.plot(x_values, normalized_median_df[col], label='Median Sale Price')

    # Plot corresponding income data for the current location
    # Assuming the income_df has the same locations as columns
    ax.plot(income_x_values, normalized_income_df["Income"].values, label='Median Income')
    
    ax.set_title(col)
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
print(median_df.columns[0])

# Calculate the average monthly change for each price column
# average_changes = {}
# for col in median_df.columns:
#     if 'MonthlyChange' in col:
#         average_changes[col] = df[col].mean()

# # Create a new DataFrame for the average monthly change rows
# average_change_row = {col: [None] for col in median_df.columns}
# for col, avg_change in average_changes.items():
#     average_change_row[col] = [avg_change]

# average_change_df = pd.DataFrame(average_change_row, index=[pd.NaT])

# # Insert the average monthly change row at the beginning of the original DataFrame
# median_df = pd.concat([average_change_df, median_df]).reset_index(drop=True)

In [ ]:
median_df["Average"] = mean_df.mean(axis=1)
plt.plot(median_df["Average"])

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

median_df['Year'] = median_df.index.year
median_df['Month'] = median_df.index.month
median_df['Quarter'] = median_df.index.quarter

result = seasonal_decompose(median_df['Average'], model='additive', period=12)
median_df['SMA'] = result.trend
median_df['Seasonal'] = result.seasonal
median_df['SSDA'] = median_df['Average'].rolling(window=12).std()
median_df['Lag1'] = median_df['SMA'].shift(1)
median_df['Lag4'] = median_df['SMA'].shift(4)

median_df.dropna(inplace=True)

plt.figure(figsize=(10, 6))
plt.plot(median_df['Average'], label='Housing Prices')
plt.plot(median_df['SMA'], label='Smooth Average', linestyle='--')
plt.title('Housing Prices with Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

result.plot()
plt.show()

In [ ]:
# List of features to plot
features = ['Seasonal', 'SMA', 'SSDA', 'Lag1', 'Lag4']

nrows = 2
ncols = 3

# Set up the figure and axes
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 10), sharex=True)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each feature
for i, feature in enumerate(features):
    axes[i].plot(median_df.index, median_df[feature])
    axes[i].set_title(feature)
    axes[i].set_ylabel(feature)
    if i >= len(features) - ncols:
        axes[i].set_xlabel('Date')

axes[5].plot(median_df.index, median_df['Lag1'], label='Lag1')
axes[5].plot(median_df.index, median_df['Lag4'], label='Lag4')
axes[5].set_title("Lag10 comparison with Lag20")
axes[5].legend()

# Remove any empty subplots (if there are any)
# for j in range(i + 1, len(axes)):
#     fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train_data = median_df[median_df['Year'] <= 2019]
test_data = median_df[median_df['Year'] > 2019]

features = ['Year', 'Month', 'Quarter', 'Seasonal', 'SMA', 'SSDA', 'Lag1', 'Lag4']
X_train = train_data[features]
y_train = train_data['Average']
X_test = median_df[features]
y_test = median_df['Average']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

plt.figure(figsize=(14, 7))
plt.plot(train_data.index, train_data['Average'], label='Train Data')
plt.plot(median_df.index, y_test, label='Test Data')
plt.plot(median_df.index, y_pred, label='Predicted Data', linestyle='--')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Average')
plt.title('Actual vs Predicted Values')
plt.show()

In [ ]:
# from geopy.geocoders import Nominatim
# import folium
# from folium.plugins import HeatMap

# geolocator = Nominatim(user_agent="geoapiExercises")

# print("AHHH")
# def geocode_location(location):
#     print(location, type(location))
#     try:
#         location = geolocator.geocode(location)
#         return (location.latitude, location.longitude)
#     except:
#         return (None, None)

# print("AHHHHHHHHH")
# median_df['Coordinates'] = median_df.T.index.to_series().apply(geocode_location)
# median_df[['Latitude', 'Longitude']] = pd.DataFrame(median_df['Coordinates'].tolist(), index=median_df.index)

# print("AHHHHHHHHHHHHHHHHHHHHHHHHHHHHH")
# heatmap_data = median_df[['Latitude', 'Longitude', 'Latest_Price']].values.tolist()

# print("AH")
# map_center = [37.0902, -95.7129]
# m = folium.Map(location=map_center, zoom_start=5)

# print("ah")
# # Add heatmap to the map
# heatmap = HeatMap(heatmap_data, radius=15, blur=10, max_zoom=1)
# m.add_child(heatmap)

# print("a")
# # Save the map to an HTML file and display
# m.save('us_housing_heatmap.html')
# m

In [ ]:
# median_df["Coordinates"]

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="DataAnalysis")
location = geolocator.geocode("Los Angeles, CA")